# FLATHER Open boundary setup

#### Imports

In [1]:
import xarray as xr
import xesmf
#from HCtFlood.kara import flood_kara
#import bottleneck
import os
import numpy as np
import cftime

#### Parent Domain

In [2]:
os.chdir("/home/nma/HDD_data/vinay/archives/IOMOM5Op/")
#exp_dir = '/home/nma/mom/vinay/MOM6dev/prepro/obc/'
exp_dir = '/home/nma/HDD_data/vinay/archives/IOMOM5Op/'

!ls

20090403.ocean_diag.nc	20120403.ocean_temp.nc	20130831.ocean_vvel.nc
20090403.ocean_salt.nc	20120403.ocean_uvel.nc	caops
20090403.ocean_temp.nc	20120403.ocean_vvel.nc	monthly
20090403.ocean_uvel.nc	20130831.ocean_diag.nc	regrid_south.nc
20090403.ocean_vvel.nc	20130831.ocean_salt.nc	regrid_west.nc
20120403.ocean_diag.nc	20130831.ocean_temp.nc
20120403.ocean_salt.nc	20130831.ocean_uvel.nc


## Setting up inputs

- uvelocity and vvelocity from NIO domain simulations
- salt,temp and ssh as tracers
- grid file for the target model 
- sample output from rigid simulation

In [9]:
uu = xr.open_mfdataset("*.ocean_uvel.nc")
vv = xr.open_mfdataset("*.ocean_vvel.nc")
salt = xr.open_mfdataset("*.ocean_salt.nc")
ssh = xr.open_mfdataset("*.ocean_diag.nc").eta_t
temp = xr.open_mfdataset("*.ocean_temp.nc")
grd_p = "/home/nma/mom/MOM6dev/exps/2012_bob_obc/INPUT/horizontal_grid.nc"
#grd_p = "/home/nma/mom/MOM6dev/prepro/obc/obcmar/20120101.ocean_sfc.nc"
grid = xr.open_dataset(grd_p)
sam_op = xr.open_dataset("/home/nma/HDD_data/vinay/archives/finals/20130101.ocean_sfc.nc")



### Merging and arranging

In [10]:
ds = xr.merge([uu,vv,salt,ssh,temp])
startslice = cftime.DatetimeNoLeap(2012, 1, 1,0,0,0,0,-1,1)
endslice = cftime.DatetimeNoLeap(2014, 1, 1,0,0,0,0,-1,1)

ds_cut = ds.sel(xu_ocean=slice(65,100), yu_ocean=slice(0,28),time=slice(startslice,endslice))

ds_cut

<xarray.Dataset>
Dimensions:         (xu_ocean: 140, yu_ocean: 112, st_ocean: 58, st_edges_ocean: 59, time: 730, nv: 2, xt_ocean: 360, yt_ocean: 240)
Coordinates:
  * xu_ocean        (xu_ocean) float64 65.12 65.38 65.62 ... 99.38 99.62 99.88
  * yu_ocean        (yu_ocean) float64 0.125 0.375 0.625 ... 27.38 27.62 27.88
  * st_ocean        (st_ocean) float64 0.5 1.5 2.5 ... 3.625e+03 4.3e+03 5e+03
  * st_edges_ocean  (st_edges_ocean) float64 0.0 1.0 2.0 ... 4.65e+03 5.35e+03
  * time            (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
  * nv              (nv) float64 1.0 2.0
  * xt_ocean        (xt_ocean) float64 30.0 30.25 30.5 ... 119.2 119.5 119.8
  * yt_ocean        (yt_ocean) float64 -30.0 -29.75 -29.5 ... 29.25 29.5 29.75
Data variables:
    u               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(92, 58, 112, 140), meta=np.ndarray>
    average_T1      (time) datetime64[ns] dask.array<chunksize=(92,), meta=np.ndarray>
    average_T2      (time) datetime64[ns] dask.array<chunksize=(92,), meta=np.ndarray>
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(92,), meta=np.ndarray>
    time_bounds     (time, nv) timedelta64[ns] dask.array<chunksize=(92, 2), meta=np.ndarray>
    v               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(92, 58, 112, 140), meta=np.ndarray>
    salt            (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(92, 58, 240, 360), meta=np.ndarray>
    eta_t           (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(92, 240, 360), meta=np.ndarray>
    temp            (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(92, 58, 240, 360), meta=np.ndarray>
Attributes:
    filename:   ocean_uvel.nc
    title:      IOMV6d : Inter-annual  QuikSCAT Run
    grid_type:  regular
    grid_tile:  N/A

## southern boundary
- nyp=0 means southern end but all nxp points
- grid['x'].isel(nyp=0) ----> the values from west boundary line ( all lon points)
- grid['y'].isel(nyp=0) ----> the lat value from bottom (southboundary) boundary line (same lat points)

In [ ]:
xgrid = grid.x[::2,::2]
ygrid = grid.y[::2,::2]

south = xr.Dataset()
#south['lon'] = grid['x'].isel(nyp=0)
#south['lat'] = grid['y'].isel(nyp=0)

south['lon'] = xgrid.isel(nyp=0)
south['lat'] = ygrid.isel(nyp=0)

#lats = np.ones(len(grid['xq'].values))*grid['yq'][0].values

#south['lon'] = grid['xq']
#south['lat'] = lats

south

## Western boundary

In [12]:
west = xr.Dataset()


#south['lon'] = xgrid.isel(nyp=0)
#south['lat'] = ygrid.isel(nyp=0)

west['lon'] = xgrid.isel(nxp=0)[:50]
west['lat'] = ygrid.isel(nxp=0)[:50]

#west['lon'] = grid['x'].isel(nxp=0)
#west['lat'] = grid['y'].isel(nxp=0)
west

<xarray.Dataset>
Dimensions:  (nyp: 50)
Dimensions without coordinates: nyp
Data variables:
    lon      (nyp) float64 ...
    lat      (nyp) float64 ...

In [19]:
regrid_south = xesmf.Regridder(ds_cut.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'}), south, 'bilinear', 
                               locstream_out=True, periodic=False, filename='regrid_south.nc')

regrid_west = xesmf.Regridder(ds_cut.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'}), west, 'bilinear', 
                              locstream_out=True, periodic=False, filename='regrid_west.nc')


u_west,v_west = regrid_west(ds_cut['u']),regrid_west(ds_cut['v'])
v_south,u_south = regrid_south(ds_cut['v']),regrid_south(ds_cut['u'])


drowned_u_west,drowned_v_west = u_west.ffill(dim='nyp').ffill(dim='st_ocean'),v_west.ffill(dim='nyp').ffill(dim='st_ocean')

drowned_v_south,drowned_u_south = v_south.ffill(dim='nxp').ffill(dim='st_ocean'),u_south.ffill(dim='nxp').ffill(dim='st_ocean')

In [23]:
drowned_v_west

<xarray.DataArray (time: 730, st_ocean: 58, nyp: 50)>
dask.array<push, shape=(730, 58, 50), dtype=float32, chunksize=(515, 58, 50), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5 1.5 2.5 3.5 ... 3.625e+03 4.3e+03 5e+03
  * time      (time) object 2012-01-01 12:00:00 ... 2013-12-31 12:00:00
    lon       (nyp) float64 77.0 77.0 77.0 77.0 77.0 ... 77.0 77.0 77.0 77.0
    lat       (nyp) float64 4.0 4.076 4.152 4.228 ... 7.496 7.572 7.648 7.724
Dimensions without coordinates: nyp
Attributes:
    regrid_method:  bilinear